In [3]:
import pandas as pd
import minsearch
import os
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'minsearch'

## Loading data

In [ ]:
df = pd.read_csv("../data/qa_data.csv")

## Ingestion test

In [ ]:
load_dotenv()

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [ ]:
qas = df.to_dict(orient='records')

In [ ]:
index = minsearch.Index(
    text_fields=[ 
                    'question_type',
                    'question'
                    'answer'],
    keyword_fields=["id"]
) 

In [2]:
index.fit(documents)

NameError: name 'index' is not defined